In [1]:
import quilt3
import pandas as pd
import numpy as np
import geopandas as gpd
import urbanaccess as ua
import access
import pandana as pdna

download OSM data from our quilt bucket

In [2]:
p = quilt3.Package.browse("osm/metro_networks_8k", "s3://spatial-ucr")

In [3]:
p['40140.h5'].fetch(dest="../data/")

Copying: 100%|██████████| 42.1M/42.1M [00:04<00:00, 10.1MB/s]  


PackageEntry('file:///Users/knaaptime/projects/healthacc/data/')

In [4]:
# read in the pre-saved OSM network using pandana

osm_network = pdna.Network.from_hdf5("../data/40140.h5")

In [5]:
# read in GTFS data from the dir extracted from the zip

loaded_feeds = ua.gtfsfeed_to_df(gtfsfeed_path="/Users/knaaptime/projects/healthacc/data/rside_gtfs/",
                                           validation=False,
                                           verbose=True,
                                           append_definitions=False)

GTFS text file header whitespace check completed. Took 0.06 seconds
--------------------------------
Processing GTFS feed: 
The unique agency id: riverside_transit_agency was generated using the name of the agency in the agency.txt file.
Unique agency id operation complete. Took 0.01 seconds
Unique GTFS feed id operation complete. Took 0.00 seconds
Appended route type to stops
Appended route type to stop_times
--------------------------------
Successfully converted ['departure_time'] to seconds past midnight and appended new columns to stop_times. Took 0.23 seconds
1 GTFS feed file(s) successfully read as dataframes:
     
     Took 0.54 seconds


In [6]:
# transform the raw gtfs data into a transit network

ua.create_transit_net(gtfsfeeds_dfs=loaded_feeds,
                                   day='monday',
                                   timerange=['07:00:00', '10:00:00'],
                                   calendar_dates_lookup=None)

Using calendar to extract service_ids to select trips.
1 service_ids were extracted from calendar
966 trip(s) 51.58 percent of 1,873 total trip records were found in calendar for GTFS feed(s): ['']
NOTE: If you expected more trips to have been extracted and your GTFS feed(s) have a calendar_dates file, consider utilizing the calendar_dates_lookup parameter in order to add additional trips based on information inside of calendar_dates. This should only be done if you know the corresponding GTFS feed is using calendar_dates instead of calendar to specify service_ids. When in doubt do not use the calendar_dates_lookup parameter.
966 of 1,873 total trips were extracted representing calendar day: monday. Took 0.02 seconds
There are no departure time records missing from trips following monday schedule. There are no records to interpolate.
Difference between stop times has been successfully calculated. Took 0.19 seconds
Stop times from 07:00:00 to 10:00:00 successfully selected 8,613 records

In [7]:
urbanaccess_net = ua.ua_network

In [8]:
urbanaccess_net.transit_edges

,node_id_from,node_id_to,weight,unique_agency_id,unique_trip_id,sequence,id,route_type,unique_route_id,net_type
0,749_riverside_transit_agency,714_riverside_transit_agency,1.000000,riverside_transit_agency,534170020_riverside_transit_agency,1,534170020_riverside_transit_agency_1,3,16_riverside_transit_agency,transit
1,714_riverside_transit_agency,771_riverside_transit_agency,0.533333,riverside_transit_agency,534170020_riverside_transit_agency,2,534170020_riverside_transit_agency_2,3,16_riverside_transit_agency,transit
2,771_riverside_transit_agency,715_riverside_transit_agency,0.433333,riverside_transit_agency,534170020_riverside_transit_agency,3,534170020_riverside_transit_agency_3,3,16_riverside_transit_agency,transit
3,715_riverside_transit_agency,717_riverside_transit_agency,0.816667,riverside_transit_agency,534170020_riverside_transit_agency,4,534170020_riverside_transit_agency_4,3,16_riverside_transit_agency,transit
4,717_riverside_transit_agency,772_riverside_transit_agency,0.633333,riverside_transit_agency,534170020_riverside_transit_agency,5,534170020_riverside_transit_agency_5,3,16_riverside_transit_agency,transit
...,...,...,...,...,...,...,...,...,...,...
8380,7_riverside_transit_agency,549_riverside_transit_agency,11.483333,riverside_transit_agency,538597020_riverside_transit_agency,6,538597020_riverside_transit_agency_6,3,204_riverside_transit_agency,transit
8381,549_riverside_transit_agency,550_riverside_transit_agency,3.516667,riverside_transit_agency,538597020_riverside_transit_agency,7,538597020_riverside_transit_agency_7,3,204_riverside_transit_agency,transit
8382,1037_riverside_transit_agency,1100_riverside_transit_agency,0.400000,riverside_transit_agency,538603020_riverside_transit_agency,1,538603020_riverside_transit_agency_1,3,204_riverside_transit_agency,transit
8383,1100_riverside_transit_agency,1038_riverside_transit_agency,17.000000,riverside_transit_agency,538603020_riverside_transit_agency,2,538603020_riverside_transit_agency_2,3,204_riverside_transit_agency,transit


In [9]:
ua_osm = ua.create_osm_net(osm_edges=osm_network.edges_df,
                              osm_nodes=osm_network.nodes_df,
                              travel_speed_mph=3)

Created OSM network with travel time impedance using a travel speed of 3 MPH. Took 0.02 seconds


In [10]:
ua.integrate_network(urbanaccess_network=urbanaccess_net,
                             headways=False)

Loaded UrbanAccess network components comprised of:
     Transit: 2,415 nodes and 8,385 edges;
     OSM: 486,514 nodes and 742,113 edges
Connector edges between the OSM and transit network nodes successfully completed. Took 1.25 seconds
Edge and node tables formatted for Pandana with integer node ids: id_int, to_int, and from_int. Took 2.91 seconds
Network edge and node network integration completed successfully resulting in a total of 488,929 nodes and 755,328 edges:
     Transit: 2,415 nodes 8,385 edges;
     OSM: 486,514 nodes 742,113 edges; and
     OSM/Transit connector: 4,830 edges.


In [11]:
# I think somewhere ua has hardcoded the paths to be `logs` inside the cwd, so you need to move up *two* levels
ua.save_network(urbanaccess_network=urbanaccess_net,
                        filename='../../data/combined_net.h5',
                        overwrite_key = True)

Using existing data/../../data/combined_net.h5 hdf5 store.
Existing edges overwritten in data/../../data/combined_net.h5 hdf5 store.
Using existing data/../../data/combined_net.h5 hdf5 store.
Existing nodes overwritten in data/../../data/combined_net.h5 hdf5 store.


In [12]:
urbanaccess_net.net_edges.from_int.dtype

dtype('float64')

In [13]:
combined_net = pdna.Network(urbanaccess_net.net_nodes["x"],
                               urbanaccess_net.net_nodes["y"],
                               urbanaccess_net.net_edges["from_int"],
                               urbanaccess_net.net_edges["to_int"],
                               urbanaccess_net.net_edges[["weight"]],
                               twoway=False)


ValueError: Buffer dtype mismatch, expected 'long' but got 'double'